<a href="https://colab.research.google.com/github/chrishg23-jpg/HES-benchmark/blob/main/HES2full10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# HES_28.0_RESONANCE_HORIZON_TEST.py: Act XXVIII - Mapping the Range

import numpy as np
import time
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
np.random.seed(42)

# --- 1. CORE PHYSICS FUNCTIONS (Necessary for all runs) ---

def von_neumann_entropy(subsystem):
    """Computes von Neumann entropy."""
    flat = subsystem.flatten()
    rho = np.outer(flat, flat)
    rho /= np.trace(rho + 1e-10)
    eigenvalues = np.linalg.eigvalsh(rho)
    eigenvalues = eigenvalues[eigenvalues > 1e-10]
    return -np.sum(eigenvalues * np.log2(eigenvalues)) if len(eigenvalues) > 0 else 0.0

def laplacian_2d_numpy(field):
    """Optimized Laplacian (∇²S)."""
    S_padded = np.pad(field, 1, mode='wrap')
    S_center = S_padded[1:-1, 1:-1]
    L = S_padded[2:, 1:-1] + S_padded[:-2, 1:-1] + S_padded[1:-1, :-2] + S_padded[1:-1, 2:] - 4 * S_center
    return L

# --- 2. FIXED OPTIMAL PARAMETERS ---
N_A = 100
DELTA_A = 4.94      # Source Delta (Stable)
ALPHA = 3.00
BETA, GAMMA = 0.1, 0.001
STEPS = 200         # Run time
SUBSYSTEM_SIZE = 3
ZETA = 0.005        # Entropic Coupling Constant (ζ)

# Receiver Parameters (Same unstable settings for all trials)
DELTA_B = 4.99
RECEIVER_DIMENSIONS = [105, 110, 120] # Trials

# --- 3. COUPLED SIMULATION CORE ---
def run_resonance_trial(n_receiver):

    # Dimensional Scaling Factors
    L_SQUARE_A = (N_A / 100.0)**2
    L_SQUARE_B = (n_receiver / 100.0)**2

    # Initialize grids
    s_A = np.random.uniform(-1, 1, (N_A, N_A))
    s_B = np.random.uniform(-1, 1, (n_receiver, n_receiver))
    G_eff_B_history = []

    for step in range(STEPS):
        # 1. Source Grid A (Optimal Lock)
        # Compute Entropy Map A (simplified to local window for speed)
        entropy_map_A = np.zeros((N_A, N_A))
        for i in range(N_A):
            for j in range(N_A):
                # Simplified subsystem calculation for A (to speed up coupled run)
                sub_A = s_A[max(0, i-1):min(N_A, i+2), max(0, j-1):min(N_A, j+2)]
                entropy_map_A[i,j] = von_neumann_entropy(sub_A)

        curvature_A = laplacian_2d_numpy(entropy_map_A)
        mean_curvature_A = np.mean(np.abs(curvature_A))

        # Update Grid A (Source)
        nabla_s_A = (s_A - (np.roll(s_A, 1, axis=0) + np.roll(s_A, -1, axis=0) + np.roll(s_A, 1, axis=1) + np.roll(s_A, -1, axis=1)) / 4.0)
        curvature_norm_A = curvature_A / L_SQUARE_A
        s_A = s_A - (ALPHA * nabla_s_A + BETA * s_A + GAMMA * np.mean(s_A) * s_A + DELTA_A * curvature_norm_A)
        s_A = np.clip(s_A, -1, 1)

        # 2. Receiver Grid B (Unstable Dynamics)
        # Compute Entropy Map B
        entropy_map_B = np.zeros((n_receiver, n_receiver))
        for i in range(n_receiver):
            for j in range(n_receiver):
                sub_B = s_B[max(0, i-1):min(n_receiver, i+2), max(0, j-1):min(n_receiver, j+2)]
                entropy_map_B[i,j] = von_neumann_entropy(sub_B)

        curvature_B = laplacian_2d_numpy(entropy_map_B)
        coupling_term = ZETA * mean_curvature_A # The Entropic Echo

        # Update Grid B (Receiver)
        nabla_s_B = (s_B - (np.roll(s_B, 1, axis=0) + np.roll(s_B, -1, axis=0) + np.roll(s_B, 1, axis=1) + np.roll(s_B, -1, axis=1)) / 4.0)
        curvature_norm_B = curvature_B / L_SQUARE_B

        # The Coupled Equation: + coupling_term attempts stabilization
        s_B_new = s_B - (ALPHA * nabla_s_B + BETA * s_B + GAMMA * np.mean(s_B) * s_B + DELTA_B * curvature_norm_B) + coupling_term
        s_B = np.clip(s_B_new, -1, 1)

        # 3. Log G_eff of Receiver Grid B
        boundary_area = 4 * n_receiver
        boundary_entropy = np.mean([entropy_map_B[0,:], entropy_map_B[-1,:], entropy_map_B[:,0], entropy_map_B[:,-1]])
        G_eff_B = boundary_area / (4 * boundary_entropy) if boundary_entropy > 0 else 1.0e+20
        G_eff_B_history.append(G_eff_B)

    # Final G_eff is the mean of the last 50 steps
    final_G_eff_B = np.mean(G_eff_B_history[-50:])

    return final_G_eff_B

# --- 4. EXECUTION LOOP ---
results = {}
print(f"Starting HES 28.0 Resonance Horizon Test (ζ={ZETA}).")
start_time_total = time.time()
for N_R in RECEIVER_DIMENSIONS:
    start_time_trial = time.time()
    final_G_eff = run_resonance_trial(N_R)
    elapsed_time_trial = time.time() - start_time_trial

    results[N_R] = final_G_eff
    print(f"Trial N_R={N_R}: G_eff={final_G_eff:.2e} | Time={elapsed_time_trial:.2f}s")

elapsed_time_total = time.time() - start_time_total
print(f"\nResonance Horizon Sweep Finished in {elapsed_time_total:.2f} seconds.")

print("\n" + "="*60)
print("ACT XXVIII: RESONANCE HORIZON TEST REPORT")
print("============================================================")
for N_R, G_eff in results.items():
    status = "STABLE (Target Range)" if G_eff < 100 else "FAILURE (Collapse)"
    print(f"Receiver N={N_R}: G_eff = {G_eff:.2e} -> {status}")
print("-" * 60)


Starting HES 28.0 Resonance Horizon Test (ζ=0.005).
Trial N_R=105: G_eff=7.28e+11 | Time=181.03s
Trial N_R=110: G_eff=7.62e+11 | Time=186.22s
Trial N_R=120: G_eff=8.32e+11 | Time=207.12s

Resonance Horizon Sweep Finished in 574.37 seconds.

ACT XXVIII: RESONANCE HORIZON TEST REPORT
Receiver N=105: G_eff = 7.28e+11 -> FAILURE (Collapse)
Receiver N=110: G_eff = 7.62e+11 -> FAILURE (Collapse)
Receiver N=120: G_eff = 8.32e+11 -> FAILURE (Collapse)
------------------------------------------------------------
